In [6]:
pip install pandas numpy matplotlib seaborn prophet scikit-learn holidays

  Using cached prophet-1.1.6-py3-none-macosx_10_11_x86_64.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.3/932.3 kB 12.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
df = load_turnstile_data()
print(df.columns)  # 查看所有列名

成功加载2014年数据: 10810000行, 11列
成功加载2015年数据: 9950000行, 11列
成功加载2016年数据: 9492670行, 11列
成功加载2017年数据: 8880000行, 11列
成功加载2018年数据: 10111471行, 11列
合并后数据总量: 49244141行, 11列
Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date', 'Time', 'Description', 'Entries', 'Exits                                                     '], dtype='object')


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta
import os
import glob
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.offline as py
import plotly.graph_objs as go
import holidays
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
sns.set(style="whitegrid")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("开始执行地铁客流量分析与预测...")

def load_turnstile_data(years=None):
    """
    加载多年的闸机数据
    
    参数:
    years (list): 需要加载的年份列表，默认为所有可用数据(2014-2018)
    
    返回:
    DataFrame: 合并后的数据框
    """
    if years is None:
        years = ['2014', '2015', '2016', '2017', '2018']
    
    all_data = []
    desktop_path = os.path.join(os.path.expanduser('~'),'Desktop')

    for year in years:
        file_name = f'turnstile-usage-data-{year}.csv'
        file_path = os.path.join(desktop_path, file_name)
        
        if os.path.exists(f'{file_path}.csv'):
            df = pd.read_csv(f'{file_path}.csv')
        elif os.path.exists(file_path):
            df = pd.read_csv(file_path)
        elif os.path.exists(f'{file_path}.txt'):
            df = pd.read_csv(f'{file_path}.txt')
        else:
            print(f"警告: 找不到{year}年的数据文件")
            continue
        
        print(f"成功加载{year}年数据: {df.shape[0]}行, {df.shape[1]}列")
        all_data.append(df)
    
    if not all_data:
        raise FileNotFoundError("未能找到任何数据文件")
    
    df = pd.concat(all_data, ignore_index=True)
    print(f"合并后数据总量: {df.shape[0]}行, {df.shape[1]}列")
    return df

def preprocess_data(df):
    """数据预处理"""
    print("开始数据预处理...")

    df.columns = df.columns.str.strip()
    print("修正后的列名:", df.columns.tolist()) 
    
    df['Date'] = pd.to_datetime(df['Date'])

    df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
    
    df['Entries'] = pd.to_numeric(df['Entries'], errors='coerce')
    df['Exits'] = pd.to_numeric(df['Exits'], errors='coerce')

    print("计算客流量增量...")
    df = df.sort_values(by=['C/A', 'Unit', 'SCP', 'Datetime'])
    
    df['ENTRIES_DIFF'] = df.groupby(['C/A', 'Unit', 'SCP'])['Entries'].diff()
    df['EXITS_DIFF'] = df.groupby(['C/A', 'Unit', 'SCP'])['Exits'].diff()
    
    # 处理异常值
    df['ENTRIES_DIFF'] = df['ENTRIES_DIFF'].clip(lower=0)
    df['EXITS_DIFF'] = df['EXITS_DIFF'].clip(lower=0)
    
    max_reasonable_count = 10000
    df['ENTRIES_DIFF'] = df['ENTRIES_DIFF'].clip(upper=max_reasonable_count)
    df['EXITS_DIFF'] = df['EXITS_DIFF'].clip(upper=max_reasonable_count)
    
    # 添加时间特征
    df['DOW'] = df['Datetime'].dt.dayofweek  # 星期几 (0=周一, 6=周日)
    df['HOUR'] = df['Datetime'].dt.hour
    df['MONTH'] = df['Datetime'].dt.month
    df['YEAR'] = df['Datetime'].dt.year
    
    df = df.dropna(subset=['ENTRIES_DIFF', 'EXITS_DIFF'])
    
    print(f"预处理后数据量: {df.shape[0]}行")
    return df

def aggregate_data(df, freq='D'):
    """
    将数据聚合到指定频率
    
    参数:
    df (DataFrame): 预处理后的数据框
    freq (str): 聚合频率，'D'表示按天，'H'表示按小时
    
    返回:
    DataFrame: 聚合后的数据框
    """
    print(f"聚合数据到{freq}频率...")
    
    date_col = 'Date' if freq == 'D' else 'Datetime'
    
    df_agg = df.groupby([date_col, 'Station']).agg({
        'ENTRIES_DIFF': 'sum',
        'EXITS_DIFF': 'sum'
    }).reset_index()
    
    df_agg['TOTAL_TRAFFIC'] = df_agg['ENTRIES_DIFF'] + df_agg['EXITS_DIFF']
    
    print(f"聚合后数据量: {df_agg.shape[0]}行")
    return df_agg

def analyze_station_traffic(df_agg):
    """分析各站点客流量"""
    print("分析站点客流量...")
    
    station_traffic = df_agg.groupby('Station').agg({
        'TOTAL_TRAFFIC': 'sum',
        'ENTRIES_DIFF': 'sum',
        'EXITS_DIFF': 'sum'
    }).reset_index()
    
    station_traffic = station_traffic.sort_values('TOTAL_TRAFFIC', ascending=False)
    
    print("客流量最高的10个站点:")
    print(station_traffic.head(10))
    
    # 普劳特前10个最繁忙的站点
    plt.figure(figsize=(12, 8))
    top_stations = station_traffic.head(10)
    
    # 堆叠条形图
    plt.barh(top_stations['Station'], top_stations['ENTRIES_DIFF'], color='skyblue', label='进站')
    plt.barh(top_stations['Station'], top_stations['EXITS_DIFF'], left=top_stations['ENTRIES_DIFF'], 
             color='lightcoral', label='出站')
    
    plt.xlabel('客流量')
    plt.ylabel('站点')
    plt.title('纽约地铁最繁忙的10个站点')
    plt.legend()
    plt.tight_layout()
    plt.savefig('top_stations_traffic.png', dpi=300)
    plt.close()
    
    return station_traffic

def prepare_time_series(df_agg, top_n_stations=None, target_station=None):
    """
    准备时间序列数据用于Prophet模型
    
    参数:
    df_agg (DataFrame): 聚合后的数据框
    top_n_stations (int): 选取客流量最高的前N个站点，默认为None
    target_station (str): 指定分析的目标站点名称，默认为None
    
    返回:
    DataFrame: 符合Prophet要求格式的数据框
    """
    print("准备时间序列数据...")
    
    # 如果指定了目标站点
    if target_station:
        df_station = df_agg[df_agg['Station'] == target_station].copy()
        if df_station.empty:
            print(f"警告: 未找到站点 '{target_station}'，将使用总体数据")
            df_station = df_agg.copy()
    
    # 如果已经指定要选取top N站点了
    elif top_n_stations:
        # 找出客流量最高的N个站点
        top_stations = analyze_station_traffic(df_agg)['Station'].head(top_n_stations).tolist()
        df_station = df_agg[df_agg['Station'].isin(top_stations)].copy()
    
    # 否则的话使用所有站点的总和
    else:
        df_station = df_agg.copy()
        
    # 然后按日期聚合所有选中站点的客流量
    ts_data = df_station.groupby('Date').agg({
        'ENTRIES_DIFF': 'sum',
        'EXITS_DIFF': 'sum',
        'TOTAL_TRAFFIC': 'sum'
    }).reset_index()
    
    # 这是Prophet要求的格式: ds (日期) 和 y (预测目标) 好像要求时间戳格式🤔
    prophet_df = ts_data.rename(columns={'Date': 'ds', 'TOTAL_TRAFFIC': 'y'})
    
    print(f"时间序列数据范围: {prophet_df['ds'].min()} 到 {prophet_df['ds'].max()}")
    return prophet_df

def add_holidays(df):
    """添加美国假日信息到数据框"""
    print("添加假日信息...")
    
    df['ds'] = pd.to_datetime(df['ds'])

    start_year = df['ds'].min().year
    end_year = df['ds'].max().year
    
    # 获取大漂亮的假日
    us_holidays = holidays.US(years=range(start_year, end_year + 1))
    
    # 先构造一个假日数据框 集中处理
    holiday_df = pd.DataFrame(
        [(pd.Timestamp(date), name) for date, name in us_holidays.items()],
        columns=['ds', 'holiday']
    )
    
    # 添加假日前后的影响天数
    holiday_df['lower_window'] = -1  # 假日前一天
    holiday_df['upper_window'] = 1   # 假日后一天
    
    # 重要假日影响应该会大一点 就多一天吧 重要假日是我猜的（like the fourth of July~
    important_holidays = ['New Year', 'Independence Day', 'Thanksgiving', 'Christmas Day']
    for holiday in important_holidays:
        mask = holiday_df['holiday'].str.contains(holiday, case=False, na=False)
        holiday_df.loc[mask, 'lower_window'] = -2  # 重要假日前两天
        holiday_df.loc[mask, 'upper_window'] = 2   # 重要假日后两天

    print("假日数据预览:")
    print(holiday_df.head())
    print("假日数据列名:", holiday_df.columns.tolist())
    
    return holiday_df
    
def train_prophet_model(df, forecast_periods=60, holidays_df=None):
    """
    训练Prophet模型并进行预测
    
    参数:
    df (DataFrame): 符合Prophet格式的数据
    forecast_periods (int): 预测的天数
    holidays_df (DataFrame): 假日数据框
    
    返回:
    tuple: (Prophet模型, 预测结果)
    """
    print("训练Prophet模型...")
    
    # Prophet建模
    model = Prophet(
        changepoint_prior_scale=0.05,  # 控制趋势灵活性
        seasonality_prior_scale=10,    # 增强季节性
        seasonality_mode='multiplicative',  # 乘法季节性通常更适合客流量
        daily_seasonality=True,        # 启用日内季节性
        weekly_seasonality=True,       # 启用周季节性
        yearly_seasonality=True        # 启用年季节性
    )
    
    # 添加月季节性
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # 添加季度季节性
    model.add_seasonality(name='quarterly', period=91.25, fourier_order=5)
    
    # 添加假日效应
    if holidays_df is not None:
        model.add_country_holidays(country_name='US')
        model.holidays = holidays_df
    
    model.fit(df)
    
    future = model.make_future_dataframe(periods=forecast_periods)
    
    forecast = model.predict(future)
    
    print(f"模型训练完成，预测未来{forecast_periods}天")
    print("预测结果列:", forecast.columns.tolist())
    return model, forecast

def visualize_forecast(model, forecast, df, title='地铁客流量预测'):
    """可视化预测结果"""
    print("可视化预测结果...")
    
    # 普劳特预测总图 似乎普劳特不出汉语 但我已经设置过了 仍是一个bug
    fig1 = model.plot(forecast)
    plt.title(f'{title} - Tendency Chart')
    plt.xlabel('Date')
    plt.ylabel('Ridership')
    plt.tight_layout()
    plt.savefig('prophet_forecast.png', dpi=300)
    plt.close()
    
    # 普劳特组件图
    fig2 = model.plot_components(forecast)
    plt.tight_layout()
    plt.savefig('prophet_components.png', dpi=300)
    plt.close()
    
    # 最近一年的实际值与预测值对比图吧
    plt.figure(figsize=(12, 6))
    
    # 先筛选出最近一年的数据
    last_date = df['ds'].max()
    one_year_ago = last_date - pd.Timedelta(days=365)
    
    # 然后提取最近一年的实际值和预测值
    recent_actual = df[df['ds'] >= one_year_ago]
    recent_forecast = forecast[(forecast['ds'] >= one_year_ago) & (forecast['ds'] <= last_date)]
    
    # 普劳特出实际值
    plt.plot(recent_actual['ds'], recent_actual['y'], 'k.', label='Actual Ridership')
    
    # 普劳特预测值及其置信区间
    plt.plot(recent_forecast['ds'], recent_forecast['yhat'], 'b-', label='Predict Ridership')
    plt.fill_between(recent_forecast['ds'], recent_forecast['yhat_lower'], recent_forecast['yhat_upper'], 
                    color='blue', alpha=0.2, label='95% Confidence Interval')
    
    plt.title(f'{title} - Trend over the Past Year')
    plt.xlabel('Date')
    plt.ylabel('Ridership')
    plt.legend()
    plt.tight_layout()
    plt.savefig('recent_year_comparison.png', dpi=300)
    plt.close()
    
    # 普劳特未来预测
    plt.figure(figsize=(12, 6))
    
    # 提取未来的预测值
    future_forecast = forecast[forecast['ds'] > last_date]
    
    # 普劳特预测值及其置信区间
    plt.plot(future_forecast['ds'], future_forecast['yhat'], 'r-', label='Future Ridership Forecast')
    plt.fill_between(future_forecast['ds'], future_forecast['yhat_lower'], future_forecast['yhat_upper'], 
                    color='red', alpha=0.2, label='95% Confidence Interval')
    
    plt.title(f'{title} - Future Ridership Forecast')
    plt.xlabel('Date')
    plt.ylabel('Ridership')
    plt.legend()
    plt.tight_layout()
    plt.savefig('future_forecast.png', dpi=300)
    plt.close()
    
    # 输出为交互式Plotly图表(保存为HTML)
    try:
        fig = plot_plotly(model, forecast)
        fig.update_layout(title=f'{title} - 交互式预测图')
        py.plot(fig, filename='interactive_forecast.html', auto_open=False)
        
        components_fig = plot_components_plotly(model, forecast)
        components_fig.update_layout(title=f'{title} - 交互式组件分解图')
        py.plot(components_fig, filename='interactive_components.html', auto_open=False)
        
        print("已生成交互式HTML可视化")
    except Exception as e:
        print(f"生成交互式图表时出错: {e}")
    
def analyze_holiday_effects(forecast):
    print("分析假日效应...")
    
    # 查找包含假日效应的列（通常以假日名称或 'holidays' 开头）
    holiday_cols = [col for col in forecast.columns if 'holidays' in col.lower() or any(h in col.lower() for h in ['new year', 'independence', 'thanksgiving', 'christmas'])]
    
    if not holiday_cols:
        print("未找到假日效应列，可能是假日未正确配置")
        return None
    
    print("找到的假日效应列:", holiday_cols)
    
    # 假设假日效应汇总在 'holidays' 列或单独的假日列
    holiday_effects = forecast[holiday_cols + ['ds']].copy()
    holiday_effects = holiday_effects[holiday_effects[holiday_cols].notna().any(axis=1)]
    
    # 如果有多个假日列，汇总总效应
    if len(holiday_cols) > 1:
        holiday_effects['holiday_effect'] = holiday_effects[holiday_cols].sum(axis=1)
    else:
        holiday_effects['holiday_effect'] = holiday_effects[holiday_cols[0]]
    
    # 按效应绝对值排序
    holiday_effects['abs_effect'] = holiday_effects['holiday_effect'].abs()
    holiday_effects = holiday_effects.sort_values('abs_effect', ascending=False)
    
    print("假日效应最显著的10个日期:")
    print(holiday_effects[['ds', 'holiday_effect', 'abs_effect']].head(10))
    
    # 可视化
    plt.figure(figsize=(12, 8))
    top_holidays = holiday_effects.head(15)
    plt.barh(top_holidays['ds'].dt.strftime('%Y-%m-%d'), 
             top_holidays['holiday_effect'], color='skyblue')
    
    plt.xlabel('Ridership Changes')
    plt.ylabel('Date')
    plt.title('The Impact of Holidays on Subway Ridership')
    plt.tight_layout()
    plt.savefig('holiday_effects.png', dpi=300)
    plt.close()
    
    return holiday_effects

def evaluate_model(df, forecast):
    """评估模型性能"""
    print("评估模型性能...")
    
    # 将预测结果与实际值合并
    evaluation = pd.merge(
        df[['ds', 'y']], 
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], 
        on='ds', 
        how='left'
    )
    
    # 计算评估指标
    evaluation['error'] = evaluation['y'] - evaluation['yhat']
    evaluation['abs_error'] = np.abs(evaluation['error'])
    evaluation['squared_error'] = evaluation['error'] ** 2
    
    # 计算MAE, RMSE, MAPE
    mae = evaluation['abs_error'].mean()
    rmse = np.sqrt(evaluation['squared_error'].mean())
    # 避免除以零
    evaluation['abs_pct_error'] = evaluation['abs_error'] / evaluation['y'].replace(0, np.nan) * 100
    mape = evaluation['abs_pct_error'].mean()
    
    print(f"平均绝对误差 (MAE): {mae:.2f}")
    print(f"均方根误差 (RMSE): {rmse:.2f}")
    print(f"平均绝对百分比误差 (MAPE): {mape:.2f}%")
    
    # 检查预测区间覆盖率
    evaluation['in_range'] = (evaluation['y'] >= evaluation['yhat_lower']) & (evaluation['y'] <= evaluation['yhat_upper'])
    coverage = evaluation['in_range'].mean() * 100
    print(f"95%置信区间覆盖率: {coverage:.2f}%")
    
    # 绘制误差直方图
    plt.figure(figsize=(10, 6))
    plt.hist(evaluation['error'], bins=50, color='skyblue', edgecolor='black')
    plt.axvline(0, color='red', linestyle='--')
    plt.xlabel('Forecast Error')
    plt.ylabel('Frequency')
    plt.title('Prediction Error Distribution')
    plt.tight_layout()
    plt.savefig('error_distribution.png', dpi=300)
    plt.close()
    
    # 绘制真实值与预测值的散点图
    plt.figure(figsize=(10, 6))
    plt.scatter(evaluation['y'], evaluation['yhat'], alpha=0.5)
    
    # 添加对角线

    max_val = max(evaluation['y'].max(), evaluation['yhat'].max())
    min_val = min(evaluation['y'].min(), evaluation['yhat'].min())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.xlabel('Actual Ridership')
    plt.ylabel('Predict Ridership')
    plt.title('Actual vs Predicted')
    plt.tight_layout()
    plt.savefig('actual_vs_predicted.png', dpi=300)
    plt.close()
    
    return {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'Coverage': coverage
    }

def main():
    """主函数"""
    try:
        # 1. 加载数据
        df = load_turnstile_data()
        
        # 2. 数据预处理
        df_processed = preprocess_data(df)
        
        # 3. 按天聚合数据
        df_daily = aggregate_data(df_processed, freq='D')
        
        # 4. 分析站点客流量
        station_traffic = analyze_station_traffic(df_daily)
        
        # 5. 选择分析方式
        # 可以修改为分析特定站点或者Top N站点
        # prophet_df = prepare_time_series(df_daily, target_station="34 ST-PENN STA")
        # prophet_df = prepare_time_series(df_daily, top_n_stations=5)
        prophet_df = prepare_time_series(df_daily)  # 分析所有站点总流量
        
        # 6. 添加假日信息
        holidays_df = add_holidays(prophet_df)
        
        # 7. 训练Prophet模型并预测
        model, forecast = train_prophet_model(prophet_df, forecast_periods=90, holidays_df=holidays_df)
        
        # 8. 可视化预测结果
        visualize_forecast(model, forecast, prophet_df, title='纽约地铁客流量预测')
        
        # 9. 分析假日效应
        holiday_effects = analyze_holiday_effects(forecast)
        
        # 10. 评估模型性能
        metrics = evaluate_model(prophet_df, forecast)
        
        print("\n分析完成! 已生成多个可视化图表。")
        
        # 11. 保存预测结果到CSV
        forecast.to_csv('prophet_forecast_results.csv', index=False)
        print("预测结果已保存到 'prophet_forecast_results.csv'")
        
    except Exception as e:
        print(f"程序执行过程中出错: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

开始执行地铁客流量分析与预测...
成功加载2014年数据: 10810000行, 11列
成功加载2015年数据: 9950000行, 11列
成功加载2016年数据: 9492670行, 11列
成功加载2017年数据: 8880000行, 11列
成功加载2018年数据: 10111471行, 11列
合并后数据总量: 49244141行, 11列
开始数据预处理...
修正后的列名: ['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date', 'Time', 'Description', 'Entries', 'Exits']
计算客流量增量...
预处理后数据量: 49236835行
聚合数据到D频率...
聚合后数据量: 486811行
分析站点客流量...
客流量最高的10个站点:
             Station  TOTAL_TRAFFIC  ENTRIES_DIFF   EXITS_DIFF
86    34 ST-PENN STA    340979966.0   181900317.0  159079649.0
21    14 ST-UNION SQ    264382991.0   141172443.0  123210548.0
84   34 ST-HERALD SQ    263445675.0   134087855.0  129357820.0
258         CANAL ST    181485176.0   101261720.0   80223456.0
64             23 ST    173562882.0   100211670.0   73351212.0
340        FULTON ST    171098318.0    95685852.0   75412466.0
160            86 ST    152828326.0    82951599.0   69876727.0
12            125 ST    145275667.0    79445954.0   65829713.0
268      CHAMBERS ST    127390072.0    7495

17:15:10 - cmdstanpy - INFO - Chain [1] start processing
17:15:12 - cmdstanpy - INFO - Chain [1] done processing


模型训练完成，预测未来90天
预测结果列: ['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper', 'Christmas Day (observed)', 'Christmas Day (observed)_lower', 'Christmas Day (observed)_upper', 'Columbus Day', 'Columbus Day_lower', 'Columbus Day_upper', 'Independence Day', 'Independence Day_lower', 'Independence Day_upper', 'Independence Day (observed)', 'Independence Day (observed)_lower', 'Independence Day (observed)_upper', 'Labor Day', 'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day', 'Martin Luther King Jr. Day_lower', 'Martin Luther King Jr. Day_upper', 'Memorial Day', 'Memorial Day_lower', 'Memorial Day_upper', "New Year's Day", "New Year's Day_lower", "New Year's Day_upper", "New Year's Day (observed)", "New Year's Day (observed)_lower", "New Year's Day (observed)_upper", 'Thanksgiving Day', 'Thanksgiving Day_lower', 'Thanksgiving Day_upper', 'Veterans Day', 'Veterans Day_lower', 'Veterans Day_upper'

NameError: name 'forecast' is not defined

In [44]:
import pandas as pd

# 设置 CSV 文件路径
csv_file = "/Users/dl/Desktop/turnstile-usage-data-2014.csv"  # 替换为你的 CSV 文件路径

# 读取 CSV 文件
df = pd.read_csv(csv_file)

# 确保第3列（C列）存在
if df.shape[1] < 3:
    raise ValueError("CSV file does not have at least 3 columns!")

# 获取第3列（C列，索引为2）的所有唯一值并存入列表
column_c = df.iloc[:, 2]  # 第3列
unique_values_list = column_c.unique().tolist()

# 步骤 1：打印唯一值列表
print("C列的所有唯一值（存入列表）：")
print(unique_values_list)

# 步骤 2：根据C列内容匹配并计算行数
print("\nC列各值的行数：")
value_counts = column_c.value_counts()
for value, count in value_counts.items():
    print(f"值: {value}, 行数: {count}")

print(len(unique_values_list))



C列的所有唯一值（存入列表）：
['02-00-00', '02-00-01', '02-03-00', '02-03-01', '02-03-02', '02-03-03', '02-03-04', '02-03-05', '02-03-06', '02-04-00', '02-05-00', '02-05-01', '02-06-00', '02-07-00', '02-07-01', '00-00-00', '00-00-01', '00-00-02', '00-00-03', '00-00-04', '00-03-00', '00-03-01', '00-03-02', '00-04-00', '00-07-00', '00-07-01', '00-07-02', '01-04-00', '01-05-00', '01-05-01', '01-06-00', '01-06-01', '01-06-02', '01-06-03', '01-07-00', '00-00-05', '00-00-06', '00-00-07', '01-00-00', '01-00-01', '01-00-02', '01-00-03', '01-00-04', '01-00-05', '01-04-01', '01-07-01', '01-03-00', '01-03-01', '01-03-02', '02-00-02', '02-06-01', '03-00-00', '03-00-01', '03-00-02', '03-04-00', '03-05-00', '03-06-00', '03-06-01', '03-07-00', '01-00-06', '01-00-07', '01-04-02', '01-07-02', '01-03-03', '00-04-01', '02-00-03', '02-00-04', '02-00-05', '02-04-01', '03-03-00', '03-03-01', '03-03-02', '03-04-01', '03-06-02', '03-07-01', '05-00-00', '05-00-01', '05-00-02', '05-00-03', '05-07-00', '05-07-01', '05-07-02',

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 1. 数据加载与预处理
def load_turnstile_data(file_paths):
    """
    加载MTA地铁闸机数据并合并
    """
    all_data = []
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        all_data.append(df)
    
    # 合并所有数据集
    df_combined = pd.concat(all_data, ignore_index=True)
    
    return df_combined

def preprocess_data(df):
    """
    预处理MTA地铁闸机数据
    """
    df.columns = df.columns.str.strip()
    print("修正后的列名:", df.columns.tolist())
    
    # 转换日期和时间为datetime对象
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    
    # 排序数据
    df = df.sort_values(by=['C/A', 'Unit', 'SCP', 'Datetime'])
    
    # 计算每个闸机每个时间段的实际进出站人数（差值）
    df['PREV_ENTRIES'] = df.groupby(['C/A', 'Unit', 'SCP'])['Entries'].shift(1)
    df['PREV_EXITS'] = df.groupby(['C/A', 'Unit', 'SCP'])['Exits'].shift(1)
    
    # 计算进出站人数差值
    df['ENTRIES_DIFF'] = df['Entries'] - df['PREV_ENTRIES']
    df['EXITS_DIFF'] = df['Exits'] - df['PREV_EXITS']
    
    # 处理异常值（负值或过大值）
    df.loc[df['ENTRIES_DIFF'] < 0, 'ENTRIES_DIFF'] = 0
    df.loc[df['EXITS_DIFF'] < 0, 'EXITS_DIFF'] = 0
    df.loc[df['ENTRIES_DIFF'] > 5000, 'ENTRIES_DIFF'] = 0
    df.loc[df['EXITS_DIFF'] > 5000, 'EXITS_DIFF'] = 0
    
    # 删除缺失值
    df = df.dropna(subset=['ENTRIES_DIFF', 'EXITS_DIFF'])
    
    # 提取日期特征
    df['Date'] = pd.to_datetime(df['Date'])
    df['Hour'] = df['Datetime'].dt.hour
    df['Day'] = df['Datetime'].dt.day
    df['Month'] = df['Datetime'].dt.month
    df['Year'] = df['Datetime'].dt.year
    df['Dayofweek'] = df['Datetime'].dt.dayofweek  # 0=周一，6=周日
    
    return df

def aggregate_by_station(df, time_interval='D'):
    """
    按车站和时间间隔聚合数据
    time_interval: 'D'为日，'H'为小时
    """
    # 设置时间索引后重采样
    if time_interval == 'D':
        # 日聚合
        df_agg = df.groupby(['Station', pd.Grouper(key='Datetime', freq='D')])[['ENTRIES_DIFF', 'EXITS_DIFF']].sum().reset_index()
    elif time_interval == 'H':
        # 小时聚合
        df_agg = df.groupby(['Station', pd.Grouper(key='Datetime', freq='H')])[['ENTRIES_DIFF', 'EXITS_DIFF']].sum().reset_index()
    
    # 计算总流量（进站+出站）
    df_agg['TOTAL_FLOW'] = df_agg['ENTRIES_DIFF'] + df_agg['EXITS_DIFF']
    
    return df_agg

# 2. 特定车站数据提取和特征工程
def prepare_station_data(df_agg, station_name, target_col='TOTAL_FLOW'):
    """
    准备特定车站的数据用于时间序列预测
    """
    # 提取特定车站数据
    station_data = df_agg[df_agg['Station'] == station_name].copy()
    
    # 确保数据时间上是连续的
    station_data = station_data.set_index('Datetime').sort_index()
    
    # 重采样以确保没有缺失的时间点
    if 'D' in station_data.index.freq or station_data.index.freq is None:
        station_data = station_data.resample('D').sum()
    else:
        station_data = station_data.resample('H').sum()
    
    # 线性插值填充缺失值
    station_data = station_data.interpolate(method='linear')
    
    # 提取要预测的目标列
    ts_data = station_data[[target_col]].copy()
    
    # 添加时间特征
    ts_data['hour'] = ts_data.index.hour
    ts_data['day'] = ts_data.index.day
    ts_data['month'] = ts_data.index.month
    ts_data['year'] = ts_data.index.year
    ts_data['dayofweek'] = ts_data.index.dayofweek
    ts_data['is_weekend'] = (ts_data.index.dayofweek >= 5).astype(int)
    
    return ts_data

# 3. 构建Prophet模型
def build_prophet_model(data, target_col='TOTAL_FLOW', test_size=0.1):
    """
    构建Prophet模型用于预测趋势和季节性成分
    """
    # 准备Prophet所需的数据格式
    prophet_data = data.reset_index()
    prophet_data = prophet_data.rename(columns={'DATETIME': 'ds', target_col: 'y'})
    
    # 划分训练集和测试集
    train_size = int(len(prophet_data) * (1 - test_size))
    train_data = prophet_data.iloc[:train_size].copy()
    test_data = prophet_data.iloc[train_size:].copy()
    
    # 创建并训练Prophet模型
    model = Prophet(
        changepoint_prior_scale=0.05,  # 控制趋势变化的灵活性
        seasonality_prior_scale=10,    # 控制季节性强度
        seasonality_mode='additive',   # 加法季节性
        daily_seasonality=True,        # 启用日季节性
        weekly_seasonality=True,       # 启用周季节性
        yearly_seasonality=True        # 启用年季节性
    )
    
    # 添加美国假日
    model.add_country_holidays(country_name='US')
    
    # 训练模型
    model.fit(train_data)
    
    # 预测整个数据集
    future = model.make_future_dataframe(periods=len(test_data), freq='D')
    forecast = model.predict(future)
    
    # 提取训练集的拟合结果和残差
    train_forecast = forecast.iloc[:train_size].copy()
    train_data['yhat'] = train_forecast['yhat'].values
    train_data['residual'] = train_data['y'] - train_data['yhat']
    
    # 提取测试集的预测结果
    test_forecast = forecast.iloc[train_size:].copy()
    test_data['yhat'] = test_forecast['yhat'].values
    
    # 合并数据
    full_data = pd.concat([train_data, test_data], ignore_index=True)
    
    # 返回模型、预测结果和原始数据（包含残差）
    return model, forecast, full_data

# 4. 构建LSTM模型用于预测残差
def create_lstm_dataset(data, lookback=7):
    """
    为LSTM准备时间序列数据（滑动窗口）
    """
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i])
        y.append(data[i])
    return np.array(X), np.array(y)

def build_lstm_model(data, residual_col='residual', test_size=0.1, lookback=7):
    """
    构建LSTM模型用于预测残差序列
    """
    # 确保数据是按时间排序的
    data = data.sort_values('ds')
    
    # 获取残差序列
    residuals = data[residual_col].values.reshape(-1, 1)
    
    # 数据归一化
    scaler = MinMaxScaler(feature_range=(-1, 1))
    residuals_scaled = scaler.fit_transform(residuals)
    
    # 创建LSTM数据集
    X, y = create_lstm_dataset(residuals_scaled, lookback=lookback)
    
    # 划分训练集和验证集
    train_size = int(len(X) * (1 - test_size))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # 构建LSTM模型
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(lookback, 1), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # 设置早停机制以防止过拟合
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    # 训练模型
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # 预测
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    
    # 反向转换回原始尺度
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    y_train_inv = scaler.inverse_transform(y_train)
    y_test_inv = scaler.inverse_transform(y_test)
    
    # 准备结果
    lstm_results = {
        'model': model,
        'scaler': scaler,
        'train_predict': train_predict,
        'test_predict': test_predict,
        'y_train': y_train_inv,
        'y_test': y_test_inv,
        'lookback': lookback,
        'history': history
    }
    
    return lstm_results

# 5. 混合Prophet-LSTM模型
def hybrid_forecast(prophet_forecast, lstm_results, prophet_data, test_size=0.1, weights=None):
    """
    融合Prophet和LSTM预测结果
    weights: 如果提供，使用固定权重；否则，根据验证集表现确定最优权重
    """
    # 准备Prophet预测和实际值
    train_size = int(len(prophet_data) * (1 - test_size))
    
    # 分离训练集和测试集
    train_data = prophet_data.iloc[:train_size].copy()
    test_data = prophet_data.iloc[train_size:].copy()
    
    # 获取LSTM预测的残差
    lstm_train_residuals = lstm_results['train_predict'].flatten()
    lstm_test_residuals = lstm_results['test_predict'].flatten()
    
    # 对齐索引（由于LSTM需要lookback，会减少部分样本）
    lookback = lstm_results['lookback']
    
    # Prophet训练集预测
    prophet_train_preds = train_data['yhat'].values[lookback:]
    
    # Prophet测试集预测
    prophet_test_preds = test_data['yhat'].values
    
    # 实际值
    train_actual = train_data['y'].values[lookback:]
    test_actual = test_data['y'].values
    
    # 确定最佳权重
    if weights is None:
        best_weight = optimize_weights(prophet_train_preds, lstm_train_residuals, train_actual)
    else:
        best_weight = weights
    
    # 融合预测结果
    hybrid_train_preds = prophet_train_preds * best_weight + (prophet_train_preds + lstm_train_residuals) * (1 - best_weight)
    hybrid_test_preds = prophet_test_preds * best_weight + (prophet_test_preds + lstm_test_residuals) * (1 - best_weight)
    
    # 计算评估指标
    prophet_train_rmse = np.sqrt(mean_squared_error(train_actual, prophet_train_preds))
    prophet_test_rmse = np.sqrt(mean_squared_error(test_actual, prophet_test_preds))
    
    hybrid_train_rmse = np.sqrt(mean_squared_error(train_actual, hybrid_train_preds))
    hybrid_test_rmse = np.sqrt(mean_squared_error(test_actual, hybrid_test_preds))
    
    hybrid_train_mae = mean_absolute_error(train_actual, hybrid_train_preds)
    hybrid_test_mae = mean_absolute_error(test_actual, hybrid_test_preds)
    
    try:
        hybrid_train_mape = mean_absolute_percentage_error(train_actual, hybrid_train_preds) * 100
        hybrid_test_mape = mean_absolute_percentage_error(test_actual, hybrid_test_preds) * 100
    except:
        # 处理可能的除零错误
        hybrid_train_mape = np.nan
        hybrid_test_mape = np.nan
    
    # 准备结果
    results = {
        'prophet_train_preds': prophet_train_preds,
        'prophet_test_preds': prophet_test_preds,
        'lstm_train_residuals': lstm_train_residuals,
        'lstm_test_residuals': lstm_test_residuals,
        'hybrid_train_preds': hybrid_train_preds,
        'hybrid_test_preds': hybrid_test_preds,
        'train_actual': train_actual,
        'test_actual': test_actual,
        'best_weight': best_weight,
        'metrics': {
            'prophet_train_rmse': prophet_train_rmse,
            'prophet_test_rmse': prophet_test_rmse,
            'hybrid_train_rmse': hybrid_train_rmse,
            'hybrid_test_rmse': hybrid_test_rmse,
            'hybrid_train_mae': hybrid_train_mae,
            'hybrid_test_mae': hybrid_test_mae,
            'hybrid_train_mape': hybrid_train_mape,
            'hybrid_test_mape': hybrid_test_mape
        }
    }
    
    return results

def optimize_weights(prophet_preds, lstm_residuals, actual, step=0.1):
    """
    优化Prophet和LSTM混合权重
    """
    best_rmse = float('inf')
    best_weight = 0.5  # 默认权重
    
    # 尝试不同的权重
    for weight in np.arange(0, 1.01, step):
        # 计算混合预测
        hybrid_preds = prophet_preds * weight + (prophet_preds + lstm_residuals) * (1 - weight)
        
        # 计算RMSE
        rmse = np.sqrt(mean_squared_error(actual, hybrid_preds))
        
        # 更新最佳权重
        if rmse < best_rmse:
            best_rmse = rmse
            best_weight = weight
    
    return best_weight

# 6. 可视化函数
def plot_prophet_components(prophet_model, forecast, station_name, output_dir=None):
    """
    绘制Prophet模型的分解组件
    """
    # Matplotlib静态图常规版本
    plt.figure(figsize=(15, 10))
    prophet_model.plot_components(forecast)
    plt.tight_layout()
    if output_dir:
        plt.savefig(os.path.join(output_dir, f"{station_name}_prophet_components.png"), dpi=300)
    plt.show()
    
    # Plotly交互装逼版
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=('Trend', 'Week seasonality', 'Annual seasonality', 'Day seasonality'),
        vertical_spacing=0.1,
        shared_xaxes=False
    )
    
    # 趋势图
    fig.add_trace(
        go.Scatter(x=forecast['ds'], y=forecast['trend'], mode='lines', name='Trend'),
        row=1, col=1
    )
    
    # 周季节性
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly = forecast[['weekly']].copy()
    weekly['day'] = forecast['ds'].dt.dayofweek
    weekly_avg = weekly.groupby('day')['weekly'].mean().reset_index()
    weekly_avg['day_name'] = weekly_avg['day'].map(lambda x: weekdays[x])
    
    fig.add_trace(
        go.Bar(x=weekly_avg['day_name'], y=weekly_avg['weekly'], name='Week seasonality', marker_color='indianred'),
        row=2, col=1
    )
    
    # 年季节性
    if 'yearly' in forecast.columns:
        yearly = forecast[['ds', 'yearly']].copy()
        yearly['month'] = yearly['ds'].dt.month
        yearly_avg = yearly.groupby('month')['yearly'].mean().reset_index()
        
        fig.add_trace(
            go.Scatter(x=yearly_avg['month'], y=yearly_avg['yearly'], mode='lines+markers', name='Annual Seasonality', line=dict(color='forestgreen')),
            row=3, col=1
        )
        fig.update_xaxes(title_text='Months', tickvals=list(range(1, 13)), row=3, col=1)
    else:
        fig.add_trace(
            go.Scatter(x=[0], y=[0], mode='markers', name='Annual seasonality is not in use', marker=dict(color='gray')),
            row=3, col=1
        )
    
    # 日季节性
    if 'daily' in forecast.columns:
        daily = forecast[['ds', 'daily']].copy()
        daily['hour'] = daily['ds'].dt.hour
        daily_avg = daily.groupby('hour')['daily'].mean().reset_index()
        
        fig.add_trace(
            go.Scatter(x=daily_avg['hour'], y=daily_avg['daily'], mode='lines+markers', name='Day seasonality', line=dict(color='royalblue')),
            row=4, col=1
        )
        fig.update_xaxes(title_text='小时', tickvals=list(range(0, 24)), row=4, col=1)
    else:
        fig.add_trace(
            go.Scatter(x=[0], y=[0], mode='markers', name='Daily seasonality is not in use', marker=dict(color='gray')),
            row=4, col=1
        )
        
    fig.update_layout(
        height=900,
        title_text=f"{station_name} - Prophet模型分解组件",
        showlegend=False
    )
    
    if output_dir:
        fig.write_html(os.path.join(output_dir, f"{station_name}_prophet_components.html"))
    
    fig.show()

def plot_hybrid_results(hybrid_results, station_name, output_dir=None, title=None):
    """
    绘制混合模型的预测结果对比
    """
    if title is None:
        title = f"{station_name} Prediction results of the prophet-LSTM model"
        
    train_actual = hybrid_results['train_actual']
    test_actual = hybrid_results['test_actual']
    prophet_train = hybrid_results['prophet_train_preds']
    prophet_test = hybrid_results['prophet_test_preds']
    hybrid_train = hybrid_results['hybrid_train_preds']
    hybrid_test = hybrid_results['hybrid_test_preds']
    
    # Matplotlib静态图版本
    plt.figure(figsize=(15, 6))
    
    # 训练集
    train_indices = np.arange(len(train_actual))
    plt.plot(train_indices, train_actual, label='Actual value (training set)', color='blue')
    plt.plot(train_indices, prophet_train, label='Prophet Prediction (Training Set)', color='green', linestyle='--')
    plt.plot(train_indices, hybrid_train, label='Prophrt-LSTM model Prediction (Training set)', color='red', linestyle='-.')
    
    # 测试集
    test_indices = np.arange(len(train_actual), len(train_actual) + len(test_actual))
    plt.plot(test_indices, test_actual, label='Actual value (test set)', color='blue')
    plt.plot(test_indices, prophet_test, label='Prophet Prediction (Test Set)', color='green', linestyle='--')
    plt.plot(test_indices, hybrid_test, label='Prophet-LSTM model prediction(Test set)', color='red', linestyle='-.')
    
    # 添加分隔线
    plt.axvline(x=len(train_actual), color='gray', linestyle='-')
    plt.text(len(train_actual) + 1, plt.ylim()[1] * 0.9, '测试集开始', fontsize=12)
    
    plt.title(title)
    plt.xlabel('Time Index')
    plt.ylabel('Ridership')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    if output_dir:
        plt.savefig(os.path.join(output_dir, f"{station_name}_hybrid_results.png"), dpi=300)
    plt.show()
    
    # Plotly交互装逼版
    # 创建日期索引
    train_dates = pd.date_range(end=pd.Timestamp.today() - pd.Timedelta(days=len(test_actual)), periods=len(train_actual), freq='D')
    test_dates = pd.date_range(start=train_dates[-1] + pd.Timedelta(days=1), periods=len(test_actual), freq='D')
    
    fig = go.Figure()
    
    # 添加实际值
    fig.add_trace(go.Scatter(
        x=list(train_dates) + list(test_dates),
        y=list(train_actual) + list(test_actual),
        mode='lines',
        name='Actual Ridership',
        line=dict(color='royalblue', width=2)
    ))
    
    # 添加Prophet预测值
    fig.add_trace(go.Scatter(
        x=list(train_dates) + list(test_dates),
        y=list(prophet_train) + list(prophet_test),
        mode='lines',
        name='Prophet Prediction',
        line=dict(color='forestgreen', width=1.5, dash='dash')
    ))
    
    # 添加混合模型预测值
    fig.add_trace(go.Scatter(
        x=list(train_dates) + list(test_dates),
        y=list(hybrid_train) + list(hybrid_test),
        mode='lines',
        name='Prophet-LSTM model Prediction',
        line=dict(color='firebrick', width=1.5)
    ))
    
    # 添加训练集/测试集分隔线
    fig.add_vline(
        x=train_dates[-1], line_width=1.5, line_dash="dash", line_color="gray",
        annotation_text="测试集开始", 
        annotation_position="top right"
    )
    
    # 更新布局
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Ridership',
        legend_title='Type of Data',
        hovermode='x unified',
        template='plotly_white',
        height=600,
        width=1200
    )
    
    # 添加范围选择器
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="7天", step="day", stepmode="backward"),
                    dict(count=1, label="1月", step="month", stepmode="backward"),
                    dict(count=6, label="6月", step="month", stepmode="backward"),
                    dict(count=1, label="1年", step="year", stepmode="backward"),
                    dict(step="all", label="ALL")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    
    if output_dir:
        fig.write_html(os.path.join(output_dir, f"{station_name}_hybrid_results.html"))
        
    fig.show()

def plot_residuals_analysis(hybrid_results, station_name, output_dir=None):
    """
    绘制残差分析图
    """
    # 提取残差
    train_residuals = hybrid_results['train_actual'] - hybrid_results['hybrid_train_preds']
    test_residuals = hybrid_results['test_actual'] - hybrid_results['hybrid_test_preds']
    all_residuals = np.concatenate([train_residuals, test_residuals])
    
    # Plotly版本
    # 1. 残差分布图
    fig_dist = go.Figure()
    fig_dist.add_trace(go.Histogram(
        x=all_residuals,
        nbinsx=50,
        opacity=0.7,
        marker_color='royalblue',
        name='Residual Distribution'
    ))
    
    # 添加正态分布拟合线
    x_range = np.linspace(min(all_residuals), max(all_residuals), 100)
    mean = np.mean(all_residuals)
    std = np.std(all_residuals)
    y_norm = np.exp(-(x_range - mean)**2 / (2 * std**2)) / (std * np.sqrt(2 * np.pi))
    y_norm = y_norm * len(all_residuals) * (max(all_residuals) - min(all_residuals)) / 50  # 缩放到直方图高度
    
    fig_dist.add_trace(go.Scatter(
        x=x_range,
        y=y_norm,
        mode='lines',
        name='Normal Distribution Fitting',
        line=dict(color='firebrick', width=2)
    ))
    
    fig_dist.update_layout(
        title=f'{station_name} - 残差分布分析',
        xaxis_title='Residual',
        yaxis_title='Frequency',
        template='plotly_white',
        height=500,
        width=900
    )
    
    if output_dir:
        fig_dist.write_html(os.path.join(output_dir, f"{station_name}_residual_distribution.html"))
    
    fig_dist.show()
    
    # 2. 残差时序图
    # 创建日期索引
    train_dates = pd.date_range(end=pd.Timestamp.today() - pd.Timedelta(days=len(test_residuals)), periods=len(train_residuals), freq='D')
    test_dates = pd.date_range(start=train_dates[-1] + pd.Timedelta(days=1), periods=len(test_residuals), freq='D')
    
    fig_ts = go.Figure()
    
    # 添加训练集残差
    fig_ts.add_trace(go.Scatter(
        x=train_dates,
        y=train_residuals,
        mode='lines',
        name='Residual of the training set',
        line=dict(color='royalblue', width=1)
    ))
    
    # 添加测试集残差
    fig_ts.add_trace(go.Scatter(
        x=test_dates,
        y=test_residuals,
        mode='lines',
        name='Residual of the test set',
        line=dict(color='firebrick', width=1)
    ))
    
    # 添加零线
    fig_ts.add_hline(
        y=0, line_width=1, line_dash="solid", line_color="black"
    )
    
    # 训练集/测试集分隔线
    fig_ts.add_vline(
        x=train_dates[-1], line_width=1, line_dash="dash", line_color="gray",
        annotation_text="测试集开始", 
        annotation_position="top right"
    )
    
    fig_ts.update_layout(
        title=f'{station_name} - 残差时序分析',
        xaxis_title='Date',
        yaxis_title='Residual',
        legend_title='Type of Data',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        width=900,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="7d", step="day", stepmode="backward"),
                    dict(count=1, label="1month", step="month", stepmode="backward"),
                    dict(count=3, label="3months", step="month", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    
    if output_dir:
        fig_ts.write_html(os.path.join(output_dir, f"{station_name}_residual_timeseries.html"))
    
    fig_ts.show()

def plot_station_traffic_heatmap(station_data, station_name, output_dir=None):
    """
    绘制车站客流量热力图（按小时和星期几）
    """
    # 确保有日期索引和必要的列
    if not isinstance(station_data.index, pd.DatetimeIndex):
        print("数据索引不是日期时间格式，无法创建热力图。")
        return
    
    # 准备热力图数据
    df_heatmap = station_data.copy()
    df_heatmap['hour'] = df_heatmap.index.hour
    df_heatmap['dayofweek'] = df_heatmap.index.dayofweek
    
    # 按小时和星期几聚合
    pivot_table = df_heatmap.pivot_table(
        values='TOTAL_FLOW', 
        index='hour',
        columns='dayofweek',
        aggfunc='mean'
    )
    
    # 重命名列以显示星期几
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thusrday', 'Friday', 'Saturday', 'Sunday']
    pivot_table.columns = days
    
    # Plotly热力图
    fig = go.Figure(data=go.Heatmap(
        z=pivot_table.values,
        x=pivot_table.columns,
        y=pivot_table.index,
        colorscale='Viridis',
        hoverongaps=False,
        colorbar=dict(title='Average Ridership')
    ))
    
    fig.update_layout(
        title=f'{station_name} - 客流量热力图 (按小时和星期)',
        xaxis_title='Week',
        yaxis_title='Hour',
        height=600,
        width=900,
        template='plotly_white'
    )
    
    if output_dir:
        fig.write_html(os.path.join(output_dir, f"{station_name}_traffic_heatmap.html"))
    
    fig.show()

def plot_monthly_trend(station_data, station_name, output_dir=None):
    """
    绘制月度客流量趋势图
    """
    # 按月聚合
    monthly_data = station_data['TOTAL_FLOW'].resample('M').sum()
    
    # Plotly图表
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=monthly_data.index,
        y=monthly_data.values,
        mode='lines+markers',
        name='月度总客流量',
        line=dict(color='royalblue', width=2),
        marker=dict(size=8)
    ))
    
    # 添加趋势线（简单移动平均）
    rolling_avg = monthly_data.rolling(window=3).mean()
    fig.add_trace(go.Scatter(
        x=rolling_avg.index,
        y=rolling_avg.values,
        mode='lines',
        name='3 Months MA',
        line=dict(color='firebrick', width=2, dash='dash')
    ))
    
    fig.update_layout(
        title=f'{station_name} - 月度客流量趋势',
        xaxis_title='MOnth',
        yaxis_title='Total Ridership',
        legend_title='指标',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        width=900
    )
    
    if output_dir:
        fig.write_html(os.path.join(output_dir, f"{station_name}_monthly_trend.html"))
    
    fig.show()

def print_metrics(hybrid_results):
    """
    打印评估指标
    """
    metrics = hybrid_results['metrics']
    print('\n预测评估指标:')
    print('-' * 50)
    print(f"Prophet训练集RMSE: {metrics['prophet_train_rmse']:.2f}")
    print(f"Prophet测试集RMSE: {metrics['prophet_test_rmse']:.2f}")
    print(f"混合模型训练集RMSE: {metrics['hybrid_train_rmse']:.2f}")
    print(f"混合模型测试集RMSE: {metrics['hybrid_test_rmse']:.2f}")
    print(f"混合模型训练集MAE: {metrics['hybrid_train_mae']:.2f}")
    print(f"混合模型测试集MAE: {metrics['hybrid_test_mae']:.2f}")
    print(f"混合模型训练集MAPE: {metrics['hybrid_train_mape']:.2f}%")
    print(f"混合模型测试集MAPE: {metrics['hybrid_test_mape']:.2f}%")
    print(f"最佳混合权重: {hybrid_results['best_weight']:.2f}")
    print('-' * 50)

# 7. 主函数
def main():
    """
    主函数，运行全部流程
    """
    # 1. 加载数据
    data_files = glob.glob(os.path.join(os.path.expanduser("~/Desktop"), "turnstile-usage-data-*.csv"))
    
    if not data_files:
        print("未找到数据文件，请确认数据文件位于桌面上。")
        return
    
    print(f"找到以下数据文件: {[os.path.basename(f) for f in data_files]}")
    
    # 加载数据
    print("正在加载数据...")
    df = load_turnstile_data(data_files)
    print(f"加载完成，共 {len(df)} 条记录。")
    
    # 2. 数据预处理
    print("正在预处理数据...")
    df_processed = preprocess_data(df)
    print(f"预处理完成，剩余 {len(df_processed)} 条有效记录。")
    
    # 3. 聚合数据（按天）
    print("正在聚合数据...")
    df_daily = aggregate_by_station(df_processed, time_interval='D')
    
    # 4. 选择一个繁忙的车站进行建模
    # 计算每个车站的总流量
    station_totals = df_daily.groupby('STATION')['TOTAL_FLOW'].sum().sort_values(ascending=False)
    top_stations = station_totals.head(10)
    
    print("\n流量最大的10个车站:")
    for i, (station, total) in enumerate(top_stations.items(), 1):
        print(f"{i}. {station}: {total:,}")
    
    # 选择流量最大的车站
    target_station = top_stations.index[0]
    print(f"\n选择 {target_station} 用于建模...")
    
    # 5. 准备特定车站数据
    station_data = prepare_station_data(df_daily, target_station)
    print(f"准备了 {len(station_data)} 天的数据用于模型训练和测试。")
    
    # 查看基本统计
    print("\n数据统计摘要:")
    print(station_data['TOTAL_FLOW'].describe())
    
    # 6. 构建Prophet模型
    print("\n正在构建Prophet模型...")
    prophet_model, prophet_forecast, prophet_data = build_prophet_model(station_data, test_size=0.2)
    
    # 7. 构建LSTM残差模型
    print("正在构建LSTM残差模型...")
    lstm_results = build_lstm_model(prophet_data, test_size=0.2, lookback=7)
    
    # 8. 混合模型预测
    print("正在融合模型结果...")
    hybrid_results = hybrid_forecast(prophet_forecast, lstm_results, prophet_data, test_size=0.2)
    
    # 9. 显示结果
    print_metrics(hybrid_results)
    
    # 10. 可视化
    print("\n生成可视化结果...")
    plot_prophet_components(prophet_model, prophet_forecast)
    plot_hybrid_results(hybrid_results, title=f"{target_station} 车站客流量预测")
    
    print("\n建模完成！")
    
    return prophet_model, prophet_forecast, lstm_results, hybrid_results

if __name__ == "__main__":
    main()

找到以下数据文件: ['turnstile-usage-data-2015.csv', 'turnstile-usage-data-2014.csv', 'turnstile-usage-data-2016.csv', 'turnstile-usage-data-2017.csv', 'turnstile-usage-data-2018.csv']
正在加载数据...
加载完成，共 49244141 条记录。
正在预处理数据...
修正后的列名: ['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date', 'Time', 'Description', 'Entries', 'Exits']
